In [25]:
# Import packages 
import pandas as pd
import numpy as np
# Set Working Directory
import os
# os.chdir(r'C:\Personal\IMM') # absolute path, using \ and r prefix
# wd = os.getcwd()

import json
from pprint import pprint
import timeit
from decimal import *
import traceback
from itertools import cycle
import itertools
import random

import functools
from balsa.matrices import read_mdf, to_mdf

from balsa.cheval import sample_from_weights

import logging
# set random seed to be used
seed = 12345
prng = 3

dirListing = 'c:\\personal\\IMM'

In [26]:
temp_df = pd.read_csv(r"c:/personal/imm/foo_nonmand.csv")

In [27]:
%%time
# batch in ggh zone numbers and add in two columns for i and j zones
ggh = pd.read_csv(os.path.join(dirListing, "GGH_zones.csv"))
if 'ggh_zone'  not in ggh:
    print("The 'ggh_zone' column does not exist.")

# prepare the dataframe to create a matrix that reflects zone to zone travel times
ggh['key'] = 0
ggh1 = ggh

# batch in SOV non toll times
ttime = pd.read_csv(r"c:/personal/IMM/2011_AM_SOV_NT_traveltime_matrix.csv", 
                    header = None, delim_whitespace=True, names = ['taz_i', 'taz_j', 'time'])

# attach times to ggh o-d pairs
ggh2 = pd.merge(ggh1, ggh, how='left', on='key')
ggh2.columns = ['taz_i', 'key', 'taz_j']
ggh2.drop('key', 1, inplace = True)
ggh2 = pd.merge(ggh2, ttime, how = 'left', on = ['taz_i', 'taz_j']).fillna(0)


Wall time: 8.77 s


In [28]:
%%time
# Transfer O-D level auto travel times
temp_df = pd.merge(temp_df, ggh2, how = 'left', on = ['taz_i', 'taz_j'])

Wall time: 1.37 s


# Reorder Tours to create Start Times 

In [29]:
def sorted_tours(vals, group):
    """
    
    """
    
    if len(vals) == 1:
        # if there is only one home-based tour then resort the indices such that the index where the
        # destination has a home becomes the last index, and then build a dataframe
        home_dest_ind = group.iloc[np.where(group['activity_j'] == 'home')].index.values
        a = (group.index.values == home_dest_ind).argsort(kind='mergesort')
        sorted_df = pd.DataFrame(group.values[a], group.index.values[a], group.columns)
          
    if len(vals) > 1:
    # only if the individual makes more than one home-based tour then run this code. 
        for i in range(0, len(vals)):
            
            first_ind = vals[i]
#             print(first_ind)
            
            if i+1 < len(vals):
#                 print(i)
                
                next_ind = vals[i+1]-1
#                 print(next_ind)
                df_collect = group.loc[first_ind:next_ind,:]
            
            else:
                
                next_ind = group.tail(1).index.item()
#                 print(next_ind)
                df_collect = group.loc[first_ind:next_ind,:]
            
            # run the sorting code. This is a little involved and here are the steps
            # First find the row index where the tour comes back home
            # Second, get all the indices that lie beyond the closed home tour. These records
            # are those that belong to the work based other tour.
            home_dest_ind = df_collect.iloc[np.where(df_collect['activity_j'] == 'home')].index.values
            beyond_home_dest = df_collect[df_collect.index >= home_dest_ind[0]]
            beyond_home_dest_ind = beyond_home_dest[beyond_home_dest.index > home_dest_ind[0]].index.values

            # Now replace the indices such that the work-based other trips are slotted in just after
            # the home-work trip purpose.
            all_ind = df_collect.index.values
            b = np.setdiff1d(all_ind, beyond_home_dest_ind)
            new_ind = np.insert(b, 1, beyond_home_dest_ind, axis=0)
            sorted_df = df_collect.reindex(new_ind)
            
    return sorted_df
    
    

In [30]:
%%time

sorted_df = {}

# select all the records that need reordering
worktours = temp_df.loc[temp_df['subtour_id'] == 0]
worktours = worktours.groupby(['hhid', 'pid']).size().reset_index(name = 'counts')

for i in range(0, len(worktours)):
    
    # enumerate over the household and person ids
    hhold_id = worktours.at[worktours.index[i], 'hhid']
    per_id = worktours.at[worktours.index[i], 'pid']
    
    # subset the main dataframe
    worktours_fix = temp_df.loc[(temp_df['hhid'] == hhold_id) & (temp_df['pid'] == per_id)]

    for name, group in worktours_fix.groupby(['hhid', 'pid']):

        # get the indices where the activity_i is home
        home_orig_ind = group.iloc[np.where(group['activity_i'] == 'home')].index.values
        vals = home_orig_ind.tolist()
        
        # run function to sort the tours
        sorted_df[name] = sorted_tours(vals, group)

# concatenate the dictionary of dataframes collected. 
# Then remove the corresponding records from the master dataframe and concatenate
# the sorted tours
sdf = pd.concat(sorted_df.values())
temp_df = temp_df[~temp_df.index.isin(sdf.index)]
temp_df = pd.concat([temp_df, sdf]).sort_values(['hhid', 'pid'])

Wall time: 17.3 s


# Converted to Function - dont run this

In [ ]:
%%time
sorted_df = {}

for name, group in worktours_fix.groupby(['hhid', 'pid']): 
    print(name)
    
    # get the indices where the activity_i is home
    home_orig_ind = group.iloc[np.where(group['activity_i'] == 'home')].index.values
    vals = home_orig_ind.tolist()
    
    if len(vals) == 1:
        # if there is only one home-based tour then resort the indices such that the index where the
        # destination has a home becomes the last index, and then build a dataframe
        home_dest_ind = group.iloc[np.where(group['activity_j'] == 'home')].index.values
        a = (group.index.values == home_dest_ind).argsort(kind='mergesort')
        sorted_df = pd.DataFrame(group.values[a], group.index.values[a], group.columns)
          
    if len(vals) > 1:
    # only if the individual makes more than one home-based tour then run this code. 
        for i in range(0, len(vals)):
            
            first_ind = vals[i]
            print(first_ind)
            
            if i+1 < len(vals):
                print(i)
                
                next_ind = vals[i+1]-1
                print(next_ind)
                df_collect = group.loc[first_ind:next_ind,:]
            
            else:
                
                next_ind = group.tail(1).index.item()
                print(next_ind)
                df_collect = group.loc[first_ind:next_ind,:]
            
            # run the sorting code. This is a little involved and here are the steps
            # First find the row index where the tour comes back home
            # Second, get all the indices that lie beyond the closed home tour. These records
            # are those that belong to the work based other tour.
            home_dest_ind = df_collect.iloc[np.where(df_collect['activity_j'] == 'home')].index.values
            beyond_home_dest = df_collect[df_collect.index >= home_dest_ind[0]]
            beyond_home_dest_ind = beyond_home_dest[beyond_home_dest.index > home_dest_ind[0]].index.values
            
            # Now replace the indices such that the work-based other trips are slotted in just after
            # the home-work trip purpose.
            all_ind = df_collect.index.values
            b = np.setdiff1d(all_ind, beyond_home_dest_ind)
            new_ind = np.insert(b, 1, beyond_home_dest_ind, axis=0)
            sorted_df = df_collect.reindex(new_ind)
            

# Now Create Start Times

### Batch in the Time Distributions that Nem created.

### Second, I have split NHB into inbound and outbound by keeping their values their same

In [31]:
time_dist = pd.read_csv(r"c:/personal/imm/start_time_distribution.csv")

In [32]:
time_dist.head(2)

,Time,HBW_outbound,HBW_inbound,HBS_outbound,HBS_inbound,HBU_outbound,HBU_inbound,HBO_outbound,HBO_inbound,HBM_outbound,HBM_inbound,HBE_outbound,HBE_inbound,NHB_outbound,NHB_inbound,WBO_outbound,WBO_inbound,pk_flag
0,0.0,1134,25390,13,5,28,862,1179,23355,279,1234,996,3625,3089,3089,819,85,0
1,0.5,285,7650,0,10,0,159,394,10024,116,782,480,1670,1183,1183,638,43,0


In [33]:
def validate_timedist():
    'add in a check that ensures a nonzero entry at transition points between time periods'
    pass

### Get the dataframe slice that corresponds to the purpose and direction

In [34]:
def start_times_np(df_array, time_dist, results, i):
    """
    This function takes the trips_hhold dataframe and samples a start time for each trip based on the purpose and direction.
    It ignores whether each trip record has been flagged by a peak/off-peak flag.
    
    Args: dataframe array, time distribution df, results, iterator
    
    Return: dataframe with start time attached 
    """
    # slice dataframe by purpose and direction and save indices to numpy array. This is useful for concatenating
    row_purp = df_array[i][10]
    row_dir = df_array[i][9]
    
    # sample start times and save to numpy array
    weight_column = row_purp + '_' + row_dir 
#     print("column to sample from %s" %weight_column)
    
    # get start time for first row 
    if i == 0:
        
        pkflag_row = df_array[i][21]
        time_dist1 = time_dist.loc[time_dist['pk_flag'] == pkflag_row]
        
        # now sample a start time
        start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=seed)
        start_time = start_time[['Time']].values
        start_time = start_time[0][0]
#         print("first start_time %s" %start_time)

        arr_time = start_time + df_array[i][24]/60
        d = df_array[i][24]/60
#         print("travel time of first row %s" %d)
#         print("arrival time of first row %s" %arr_time)
        
    else:
        # slice the time distribution dataframe by purpose and direction to account for the fact that now it must 
        # respect the arrival time of the above record i.e. if the arrival time of the above record is 10:00, then the
        # time distribution must be sliced to only have records beyond 10:00
        pkflag_row = df_array[i][21]
        arr_time = results[i-1][1]   # get results from previous row and second column
#         print("arrival time from previous row %s used for slicing the time dist df" %arr_time)
        
        time_dist1 = time_dist.loc[time_dist['pk_flag'] == pkflag_row]
        time_dist1 = time_dist1.loc[time_dist1['Time'] >= arr_time]
        
        # if the above slicing of the time distribution dataframe has left any records than sample
        if len(time_dist1)>0:        
        # now sample a start time
            start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=seed)
            start_time = start_time[['Time']].values ###
            start_time = start_time[0][0]
#             print("current start_time %s" %start_time)
        else:
            # if the slicing of the time distribution dataframe is left with no rows than grab the start times
            # from the previous row. This implies that the start times between this trip and the one before are
            # very close to each other.
            start_time = results[i-1][0]
#             print("start time borrowed from the previous trip %s" %start_time)
        
        # recompute arr_time to save to results
        arr_time = start_time + df_array[i][24]/60
        d1 = df_array[i][24]/60
#         print("travel time of current row %s" %d1)
#         print("arrival time of current row %s" %arr_time)
        
   
    return (start_time, arr_time)

In [35]:
%%time

results1 = []

for name, group in temp_df.groupby(['hhid', 'pid']):
#     print(name)
    # make array
    df_array = group.values
    nrows, ncols = df_array.shape
    results = [] 
    
    for i in range(nrows):
#         print("row %s" %i)

        times = start_times_np(df_array, time_dist, results, i)
        results.append(times)
#         print(results)
    results1.append(results)
        

Wall time: 2min 27s


In [36]:
from itertools import chain
# flatten the nested lists and then get ready for concantenating the results to the master dataframe
l = list(chain(*results1))
l_df = pd.DataFrame(l, index = temp_df.index, columns=['start_time', 'arrival_time'])

# attach the times
temp_df = pd.concat([temp_df, l_df], axis = 1)

# Informal ride pairing model. 

* A starting window is defined that decides how many trips are potential receipients i.e. if the user defines the window as 15 minutes, then all trips that have a start time of +-15 mins from a donor trip are potential receipients for a particular donor.

* In the informal ride pairing model, the only two things that matter are the distance from a receipient to a donor's zone
and then the distance between destinations of the receipient and donor.

* Not all records can be considered to be donors. The reasoning was as follows that if the trip times between an 
O-D pair are very low than there is no need to ride pair. 

* Thus, only trips whose O-D times exceed two times the time threshold are considered as donors. A time threshold basically
represents zones that are within a certain travel time from the donor zone.

* For an **informal** ride pairing model it was not considered appropriate to create a similarity index for selecting receipients to match to donors. This is because while one could think that a person belonging to a hhold to segment 5 might not want to ride pair to a hhold from segment 0. While this could be true in a **formal** ride pairing model, not so much in an app driven world of **informal** ride pairin. Further, there isn't another quanitifiable metric that could be used for generating a similarity index.

### Get all trip records that are within use defined threhold start time

### Randomly pick a record to start the process

In [37]:
%%time
# Only keep those records where the travel time between the O-D pairs is at least twice or some user defined value
# above the time it takes for a person to arrive at another's house
# travel time thresholds
time_thold = 5
catchment = 2
pair_selection_thold = 0.8
ride_pair_vehlist = list(range(4000000))

# SUBSET DATAFRAME
##################
# If the travel time divided by the buffer time is more than pair_selection_thold then consider those trips for
# ride pairing. The travel time for the other O-D pairs is not significant enough that one would want to wait 
# to create a car pool. These variables can be played with, but the current default settings are used here.
temp_df['pair_criteria'] = temp_df['time']/(temp_df['time'] + time_thold*catchment)
pair_candidates = temp_df.loc[temp_df['pair_criteria'] >= pair_selection_thold]

Wall time: 250 ms


In [38]:
pair_candidates.shape

(1652, 28)

In [39]:
def prepare_to_pair(pair_candidates, ride_pair_vehlist):
    """
    
    """
    # randomly sample a record and set a ride_pairing flag. This flag is unique and will be added to records that are
    # a good ride pairing match.
    frec = pair_candidates.sample(n = 1, replace = False)
    frec['ride_pair'] = np.random.choice(ride_pair_vehlist, replace = False)

    # get start_time of first record; start zone; and destination zone for the Donor trip
    frec_starttime = frec.at[frec.index[0],'start_time']
    frec_startzone = frec.at[frec.index[0], 'taz_i']
    frec_destzone = frec.at[frec.index[0], 'taz_j']

    # generate the window of start_times for the receipient trips
    window_time = 15
    start_time_below = frec_starttime - window_time/60
    start_time_up = frec_starttime + window_time/60
    
    return (frec, frec_startzone, frec_destzone, start_time_below, start_time_up)

In [40]:
def get_zone(ttime, frec_zone, time_thold, col_name):
    """
    
    """
    # first slice the travel time dataframe for the origin or destination zone
    ttime_slice = ttime.iloc[np.where(ttime['taz_i'].values == frec_zone)]
    # now slice the datafame to only keep those zones that are within a travel time buffer 
    # of the origin/dest zone
    ttime_slice = ttime_slice.iloc[np.where(ttime_slice['time'].values <= time_thold)]

    # now get all the origin or destination zone that is within the travel time buffer
    get_zones = ttime_slice['taz_j'].values
    get_zones_df = pd.DataFrame(get_zones, columns = [col_name])
    get_zones_df['key'] = 0
    
    return get_zones_df

In [41]:
def prepare_candidates(or_df, dt_df, pair_candidates, start_time_below, start_time_up, frec):
    """
    
    """
    # now get a full dataframe of all origins and destinations
    full_df = pd.merge(or_df, dt_df, how = 'left', on = 'key')
    full_df = full_df.loc[full_df['taz_i'] != full_df['taz_j']]

    # Only keep those trips that are within the start_time buffer.
    # get all the records that match the origin-destination interchanges. 
    # Also get rid of intrazonals.
    pair_candidates = pair_candidates.loc[(pair_candidates['start_time'] >= start_time_below) & 
                                          (pair_candidates['start_time'] <= start_time_up)]
    
    #TODO fix this code to mirror the one done for START TIMES
    ############################################################################################
    temp_df2 = pair_candidates.loc[(pair_candidates['taz_i'].isin(full_df['taz_i'])) & 
                                   (pair_candidates['taz_j'].isin(full_df['taz_j']))]
    temp_df2 = temp_df2.loc[temp_df2['taz_i'] != temp_df2['taz_j']]
    ############################################################################################

    # add in a flag to identify the records that will be matched together
    pair_flag = frec['ride_pair'].values
    temp_df2['ride_pair'] = pair_flag[0]
    
    return (temp_df2, pair_flag)

In [42]:
%%time
# generate a random number between 0 and 1. If the number exceeds 0.5 then the donor can receive 2 receipients;
# otherwise the donor can receive only 1 receipient
matching_index = []

nrows = pair_candidates.shape[0]

while len(pair_candidates) > 0:
    
    matching_df = []
    
    frec, frec_startzone, frec_destzone, start_time_below, start_time_up = prepare_to_pair(pair_candidates, ride_pair_vehlist)
#     print("Donor index %s" %frec.index.values)
    or_df = get_zone(ttime, frec_startzone, time_thold, 'taz_i')
    dt_df = get_zone(ttime, frec_destzone, time_thold, 'taz_j')
    df_to_sample, pair_flag = prepare_candidates(or_df, dt_df, pair_candidates, start_time_below, start_time_up, frec)
#     print("len to sample from %s" %len(df_to_sample))
#     print("Pair flag %s" %pair_flag)
    
    pairing_rows = np.random.rand()
#     print(pairing_rows)
    
    if len(df_to_sample) == 0:
        print("No receipient available for the donor %s" %frec.index.values)
    
    elif len(df_to_sample) == 1:
        matching_df = df_to_sample.sample(n=1).index.values
#         print("Recep index %s" %matching_df)
    
    elif len(df_to_sample) > 1 and pairing_rows >= 0.5:
        matching_df = df_to_sample.sample(n = 2).index.values
#         print("Recep index %s" %matching_df)
    
    elif len(df_to_sample) > 1 and pairing_rows < 0.5:
        matching_df = df_to_sample.sample(n = 1).index.values
#         print("Recep index %s" %matching_df)
    
    # update the pair candidates to remove before next round
    if len(df_to_sample) == 0:
        
        pair_candidates = pair_candidates[~pair_candidates.index.isin(frec.index.values)]        
    else:
        
        indices_remove = np.concatenate([matching_df, frec.index.values], axis = 0)
        pair_candidates = pair_candidates[~pair_candidates.index.isin(indices_remove)]
    
    # concatenate the indices of the receipient and donor zones; 
    # reshape the arrays; create an array of the pair_flag and
    # then bring them together.    
    if len(df_to_sample) > 0:
    
        matching_df = np.concatenate((matching_df, frec.index.values), axis=0)
        matching_df = matching_df.reshape(matching_df.shape[0], 1)
        pair_array = np.full((matching_df.shape[0], 1), pair_flag)
        matching_df = np.c_[matching_df, pair_array]    

    matching_index.append(matching_df)
    

No receipient available for the donor [1533]
No receipient available for the donor [50334]
No receipient available for the donor [41930]
No receipient available for the donor [36570]
No receipient available for the donor [43160]
No receipient available for the donor [13666]
No receipient available for the donor [9006]
No receipient available for the donor [49068]
No receipient available for the donor [155]
No receipient available for the donor [17124]
No receipient available for the donor [48435]
No receipient available for the donor [5989]
No receipient available for the donor [25688]
No receipient available for the donor [29881]
No receipient available for the donor [40720]
No receipient available for the donor [16159]
No receipient available for the donor [40292]
No receipient available for the donor [19375]
No receipient available for the donor [11260]
No receipient available for the donor [12783]
No receipient available for the donor [2593]
No receipient available for the donor [5

In [44]:
# flatten the nested lists and then get ready for concantenating the results to the master dataframe
l1 = list(chain(*matching_index))
l_df1 = pd.DataFrame(l1, columns=['ind', 'veh_num']).set_index('ind')

# attach the times
# temp_df = pd.concat([temp_df, l_df], axis = 1)

# Cobverted to Functions - dont run

In [ ]:
for i in range(0, 1):
    frec, frec_startzone, frec_destzone, start_time_below, start_time_up = prepare_to_pair(pair_candidates, ride_pair_vehlist)
    print("Donor index %s" %frec.index.values)
    or_df = get_zone(ttime, frec_startzone, time_thold, 'taz_i')
    dt_df = get_zone(ttime, frec_destzone, time_thold, 'taz_j')
    df_to_sample, pair_flag = prepare_candidates(or_df, dt_df, pair_candidates, start_time_below, start_time_up, frec)
    print("len to sample from %s" %len(df_to_sample))
    
    pairing_rows = np.random.rand()
    print(pairing_rows)
    
    if len(df_to_sample) == 0:
        print("No receipient available for the donor %s" %frec.index.values)
    
    elif len(df_to_sample) == 1:
        matching_df = df_to_sample.sample(n=1).index.values
        print("Recep index %s" %matching_df)
    
    elif len(df_to_sample) > 1 and pairing_rows >= 0.5:
        matching_df = df_to_sample.sample(n = 2).index.values
        print("Recep index %s" %matching_df)
    
    elif len(df_to_sample) > 1 and pairing_rows < 0.5:
        matching_df = df_to_sample.sample(n = 1).index.values
        print("Recep index %s" %matching_df)
    
    indices_remove = np.concatenate([matching_df, frec.index.values], axis = 0)
    pair_candidates = pair_candidates[~pair_candidates.index.isin(indices_remove)]

In [ ]:
%%time
# randomly sample a record and set a ride_pairing flag. This flag is unique and will be added to records that are
# a good ride pairing match.
frec = pair_candidates.sample(n = 1, random_state = prng, replace = False)
frec['ride_pair'] = np.random.choice(ride_pair_vehlist, replace = False)

# get start_time of first record; start zone; and destination zone for the Donor trip
frec_starttime = frec.at[frec.index[0],'start_time']
frec_startzone = frec.at[frec.index[0], 'taz_i']
frec_destzone = frec.at[frec.index[0], 'taz_j']

# generate the window of start_times for the receipient trips
window_time = 15
start_time_below = frec_starttime - window_time/60
start_time_up = frec_starttime + window_time/60
    

In [ ]:
%%time
# ORIGIN ZONE
# first slice the travel time dataframe for the origin zone
ttime_slice = ttime.iloc[np.where(ttime['taz_i'].values == frec_startzone)]
# now slice the datafame to only keep those zones that are within a travel time buffer of the origin zone
ttime_slice = ttime_slice.iloc[np.where(ttime_slice['time'].values <= time_thold)]

# now get all the origin zones that are within the travel time buffer
origin_zones = ttime_slice['taz_j'].values
or_df = pd.DataFrame(origin_zones, columns = ['taz_i'])
or_df['key'] = 0

# DESTINATION ZONE
# first slice the travel time dataframe for the destination zone
ttime_slice = ttime.iloc[np.where(ttime['taz_i'].values == frec_destzone)]

# now slice the datafame to only keep those records that are within a travel time buffer of the destination
ttime_slice = ttime_slice.iloc[np.where(ttime_slice['time'].values <= time_thold)]

# now get all the dest zones that are within the travel time buffer
dest_zones = ttime_slice['taz_j'].values
dt_df = pd.DataFrame(dest_zones, columns = ['taz_j'])
dt_df['key'] = 0


In [ ]:
%%time
# now get a full dataframe of all origins and destinations
full_df = pd.merge(or_df, dt_df, how = 'left', on = 'key')
full_df = full_df.loc[full_df['taz_i'] != full_df['taz_j']]

# Only keep those trips that are within the start_time buffer.
# get all the records that match the origin-destination interchanges. 
# Also get rid of intrazonals.
pair_candidates = pair_candidates.loc[(pair_candidates['start_time'] >= start_time_below) & 
                                      (pair_candidates['start_time'] <= start_time_up)]
temp_df2 = pair_candidates.loc[(pair_candidates['taz_i'].isin(full_df['taz_i'])) & 
                               (pair_candidates['taz_j'].isin(full_df['taz_j']))]
temp_df2 = temp_df2.loc[temp_df2['taz_i'] != temp_df2['taz_j']]

# add in a flag to identify the records that will be matched together
pair_flag = frec['ride_pair'].values
temp_df2['ride_pair'] = pair_flag[0]

In [ ]:
%%time
# generate a random number between 0 and 1. If the number exceeds 0.5 then the donor can receive 2 receipients;
# otherwise the donor can receive only 1 receipient
matching_index = []

pairing_rows = np.random.rand()
print(pairing_rows)


if pairing_rows > 0.5:
    # sample and get index of row sampled. Also inser the pair_flag to the numpy array
    matching_df = temp_df2.sample(n=2).index.values

else:
    # sample and get index of row sampled. Also inser the pair_flag to the numpy array
    matching_df = temp_df2.sample(n=1).index.values

# concatenate the indices of the receipient and donor zones; 
# reshape the arrays; create an array of the parir_flag and
# then bring them together
matching_df = np.concatenate((matching_df, frec.index.values), axis=0)
matching_df = matching_df.reshape(matching_df.shape[0], 1)
pair_array = np.full((matching_df.shape[0], 1), pair_flag)
matching_df = np.c_[matching_df, pair_array]
# print(matching_df)    

matching_index.append(matching_df)

In [ ]:
# make one unique numpy array
unique_arr = np.unique(np.concatenate((origin_zones, dest_zones), axis=0))

# convert to dataframe and get it ready for merging with master trip records
unique_arr_df = pd.DataFrame(unique_arr, columns = ['zones'] )
unique_arr_df['key'] = 0
unique_arr_df1 = unique_arr_df

unique_arr_df = pd.merge(unique_arr_df, unique_arr_df1, how = 'left', on = 'key')

In [ ]:
%load_ext line_profiler
%lprun -f start_times_round2 start_times_round2(df_array, time_dist, results, i)

# Old Code and Attempts

In [ ]:
def start_times_round1(trips_df, time_dist, purp_dir_product):
    """
    This function takes the trips_hhold dataframe and samples a start time for each trip based on the purpose and direction.
    It ignores whether each trip record has been flagged by a peak/off-peak flag.
    
    Args:
    
    Return: dataframe with start time attached 
    """
    # slice dataframe by purpose and direction and save indices to numpy array. This is useful for concatenating
    df_dir = trips_df.iloc[np.where((trips_df['purpose'].values == purp_dir_product[0]) & 
                                    (trips_df['tour_direction'].values == purp_dir_product[1]))].index.values
    
    # sample start times and save to numpy array
    weight_column = purp_dir_product[0] + '_' + purp_dir_product[1] 
    df_dir_sample = time_dist.sample(len(df_dir), weights= time_dist[weight_column], replace=True, random_state=seed)
    df_dir_sample = df_dir_sample[['Time']].values
    
    # Append the sampled times with the sliced dataframe indices
    df_dir_sample = pd.DataFrame(df_dir_sample, index=df_dir, columns=['start_time'])

    
    return df_dir_sample    
    

In [ ]:
%%time
# purp_dir_product = [('NHB', 'outbound')]
temp_df_test_pkoffpk = {}
temp_df_test_purpdir = {}
for purp_dir in purp_dir_product:
    
    for pkoff_flag in range(0,2):
        
        temp_df1 = temp_df.loc[temp_df['peak_flag'] == pkoff_flag]
        time_dist1 = time_dist.loc[time_dist['pk_flag'] == pkoff_flag]
        
        # run start times for all combinations of trip purpose and tour direction
        temp_df_test_pkoffpk[pkoff_flag] = start_times_round1(temp_df1, time_dist1, purp_dir)
        temp_df_test_pkoffpk_comb = pd.concat(temp_df_test_pkoffpk.values(), axis = 0)
    
    temp_df_test_purpdir[purp_dir] = temp_df_test_pkoffpk_comb

# concatenate all the sampled start times in the dictionary and then concatenate it back to the main dataframe
temp_df_test1 = pd.concat(temp_df_test_purpdir.values(), axis = 0)
final_df = pd.concat([temp_df, temp_df_test1], axis= 1)

In [ ]:
final_df.head()

## Now test if start times are consistent with the following:

* The subsequent trip segmented by hhid and pid must start only after the preceeding start time + travel time between the O-D pair
* If not, then the start time of the subsequent trip must be adjusted by sampling from the time distribution but after controlling for logic such as:  


### Join in O-D travel times, by auto only
#### This is made up currently, get final from GGHMV4 run

In [ ]:
%%time
# batch in ggh zone numbers and add in two columns for i and j zones
ggh = pd.read_csv(os.path.join(dirListing, "GGH_zones.csv"))
if 'ggh_zone'  not in ggh:
    print("The 'ggh_zone' column does not exist.")

# prepare the dataframe to create a matrix that reflects zone to zone travel times
ggh['key'] = 0
ggh1 = ggh

# create proxy travel times
ggh2 = pd.merge(ggh1, ggh, how='left', on='key')
ggh2.columns = ['taz_i', 'traveltime', 'taz_j']
ggh2['traveltime'] = np.random.randint(100, size = len(ggh2))

In [ ]:
%%time
# Transfer O-D level auto travel times
final_df = pd.merge(final_df, ggh2, how = 'left', on = ['taz_i', 'taz_j'])

# Add in column to reflect the start_time + travel time
final_df['arrival_time'] = final_df['start_time'] + final_df['traveltime']/60

In [ ]:
def starttime_flag(df, first_row_arrtime):
    """
    This function evaluates the rows in a dataframe and establishes whether the start time of a record is consistent
    with the arrival time of the preceeding record i.e. the starttime of the next trip should be greater than the 
    arrival time of the previous trip.
    
    Arguments: dataframe
    
    Return: list with index and flag of processed rows in dataframe 
    """

#     duration = float('%.1f' % round(df.at[df.index[i],'start_time'] - first_row_arrtime,2))
    duration = df.at[df.index[i],'start_time'] - first_row_arrtime
     
    if duration >= -0.5:
        # set to 1 if duration is fine and start_time of the subsequent time is after the arrival
        # time of the preceeding time
        flag = 1
    else:
        flag = 0
     
    ind = df.index[i]

    return (ind, flag, duration)

In [ ]:
%%time
results = []

for name, group in final_df.groupby(['hhid', 'pid']):
    
    # make dataframe to pass to function
#     gr_df = pd.DataFrame(group)
    
    for i in range(1, len(group)):
            
        # update the arrive time value and run the function
        first_row_arrtime = group.at[group.index[i-1], 'arrival_time']
        ind, flag, duration = starttime_flag(group, first_row_arrtime)
    
        # save results in list
        results.append((ind, flag, duration))

# build datafarme for appending to master df
# append the flag and duration to the master df and also fill in a dummy value for the NaN. This value is a placeholder
results_df = pd.DataFrame(np.array(results).reshape(len(results), 3), columns = (['ind', 'flag', 'duration']))
results_df.set_index('ind', inplace=True)
final_df = pd.concat([final_df, results_df], axis = 1).fillna(10)

In [ ]:
results[0][2]

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv(r"c:/personal/imm/test_starttime_flag.csv", index = False)

## Now select all those groups that had a negative duration in them. 

#### These need to be fixed

In [ ]:
%%time
# select all the records where the duration was a negative. Then group on hhid and pid.
# These unique hhid and pid values need to be re-processed with respect to start times
need_fix = final_df.iloc[np.where(final_df['duration'].values < 0)]
need_fix_gr = need_fix.groupby(['hhid', 'pid']).size().reset_index(name = 'counts')

# get all the records that need to be fixed. One cannot only keep the records that need to be fixed,
# but has to get all the records belonging to a particular hhid and pid combination to ensure consistency
keys = ['hhid', 'pid']
i1 = final_df.set_index(keys).index
i2 = need_fix_gr.set_index(keys).index
df_fix = final_df[i1.isin(i2)]   # df that needs fixing

In [ ]:
need_fix_gr.head()

In [ ]:
print(df_fix.shape)
df_fix.head(9)

In [ ]:
tt = time_dist.loc[time_dist['pk_flag'] == 1]
tt1 = tt.loc[tt['Time'] >= 7]
tt1

In [ ]:
def start_times_round2(trips_df, time_dist, purp_dir_product1, i):
    """
    This function takes the trips_hhold dataframe and samples a start time for each trip based on the purpose and direction.
    It ignores whether each trip record has been flagged by a peak/off-peak flag.
    
    Args:
    
    Return: dataframe with start time attached 
    """
    # slice dataframe by purpose and direction and save indices to numpy array. This is useful for concatenating
    df_dir = trips_df.iloc[np.where((trips_df['purpose'].values == purp_dir_product1[0]) & 
                                    (trips_df['tour_direction'].values == purp_dir_product1[1]))].index.values
    
    # sample start times and save to numpy array
    weight_column = purp_dir_product1[0] + '_' + purp_dir_product1[1] 
    
    if trips_df.at[trips_df.index[i], 'duration'] < 0:
        
        # get arrival time of previous row
        artime = trips_df.at[trips_df.index[i-1], 'arrival_time']
        
        # now subset the time distribution dataframe for the peak period of the trip and arrival 
        # time from previous record
        pkflag_row = trips_df.at[trips_df.index[i], 'peak_flag']
        time_dist2 = time_dist.loc[time_dist['pk_flag'] == pkflag_row]
        time_dist2 = time_dist2.loc[time_dist2['Time'] >= artime]
        
        # sample
        df_dir_sample = time_dist2.sample(n=1, weights= time_dist2[weight_column], replace=True, random_state=seed)
        df_dir_sample = df_dir_sample[['Time']].values
        
        return df_dir_sample 
            
    # Append the sampled times with the sliced dataframe indices
#     df_dir_sample = pd.DataFrame(df_dir_sample, index=df_dir, columns=['start_time'])

    
        

In [ ]:
# get unique values to loop over and create a list
purposes1 = df_fix['purpose'].unique().tolist()
directions1 = df_fix['tour_direction'].unique().tolist()

purp_dir_product1 = list(itertools.product(purposes1, directions1))

In [ ]:
df_fix1 = df_fix.loc[df_fix['hhid'] == 2]
df_fix1

In [ ]:
temp_df_fix_pkoffpk = {}
for purp_dir in purp_dir_product1:
        
        for i in range(0, len(df_fix1)):    

            # run start times for all combinations of trip purpose and tour direction
            temp_df_fix_pkoffpk = start_times_round2(df_fix1, time_dist, purp_dir, i)
#             temp_df_test_pkoffpk_comb = pd.concat(temp_df_test_pkoffpk.values(), axis = 0)
    
#     temp_df_test_purpdir[purp_dir] = temp_df_test_pkoffpk_comb

In [ ]:
type(temp_df_fix_pkoffpk)

In [ ]:
for purp_dir in purp_dir_product:
for i in range(0, len(need_fix_gr)):
    
    start_times_round1(temp_df1, time_dist1, purp_dir)
    
    

    n1 = need_fix_gr.at[need_fix_gr.index[i], 'hhid']
    n2 = need_fix_gr.at[need_fix_gr.index[i], 'pid']
    print(n1, n2)